In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pywt

training_path = r'MISAHUB\Dataset\training'
validation_path = r'MISAHUB\Dataset\validation'
num_images_per_class = 300
classes = ['Angioectasia\images', 'Bleeding\images', 'Erosion\images', 'Erythema\images', 
           'Foreign Body\images', 'Lymphangiectasia\images', 'Normal\images', 'Polyp\images', 
           'Ulcer\images', 'Worms\images']

def compute_wavelet_transform_rgb(image_path):
    if not os.path.exists(image_path):
        print(f"File does not exist: {image_path}")
        return None
    
    image_bgr = cv2.imread(image_path)
    
    if image_bgr is None:
        print(f"Failed to load image: {image_path}")
        return None
    
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    
    features = []
    for i in range(3):
        channel = image_rgb[:, :, i]
        coeffs = pywt.wavedec2(channel, 'db1', level=3)
        coeffs_flat = [coeffs[0].flatten()]
        coeffs_flat += [c.flatten() for c in coeffs[1]]
        features.append(np.concatenate(coeffs_flat))
    
    return np.concatenate(features)

def load_images_and_labels(data_path, num_images_per_class):
    data = []
    labels = []
    
    for label, class_name in enumerate(classes):
        class_path = os.path.join(data_path, class_name)
        
        if not os.path.exists(class_path):
            print(f"Class folder does not exist: {class_path}")
            continue
        
        images = os.listdir(class_path)[:num_images_per_class]
        for img in images:
            img_path = os.path.join(class_path, img)
            features = compute_wavelet_transform_rgb(img_path)
            
            if features is not None:
                data.append(features)
                labels.append(label)
    
    return np.array(data), np.array(labels)

X_train, y_train = load_images_and_labels(training_path, num_images_per_class)
X_test, y_test = load_images_and_labels(validation_path, num_images_per_class)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

training_path = r'MISAHUB\Dataset\training'
validation_path = r'MISAHUB\Dataset\validation'
num_images_per_class = 300
classes = ['Angioectasia\images', 'Bleeding\images', 'Erosion\images', 'Erythema\images', 
           'Foreign Body\images', 'Lymphangiectasia\images', 'Normal\images', 
           'Polyp\images', 'Ulcer\images', 'Worms\images']

def compute_color_histogram(image_path):
    if not os.path.exists(image_path):
        print(f"File does not exist: {image_path}")
        return None
    
    image_bgr = cv2.imread(image_path)
    
    if image_bgr is None:
        print(f"Failed to load image: {image_path}")
        return None
    
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    hist = []
    for i in range(3):
        hist_channel = cv2.calcHist([image_rgb], [i], None, [256], [0, 256])
        hist.append(hist_channel.flatten())
    return np.concatenate(hist)

def load_images_and_labels(data_path, num_images_per_class):
    data = []
    labels = []
    
    for label, class_name in enumerate(classes):
        class_path = os.path.join(data_path, class_name)
        
        if not os.path.exists(class_path):
            print(f"Class folder does not exist: {class_path}")
            continue
        
        images = os.listdir(class_path)[:num_images_per_class]
        for img in images:
            img_path = os.path.join(class_path, img)
            hist = compute_color_histogram(img_path)
            
            if hist is not None:
                data.append(hist)
                labels.append(label)
    
    return np.array(data), np.array(labels)

X_train, y_train = load_images_and_labels(training_path, num_images_per_class)
X_test, y_test = load_images_and_labels(validation_path, num_images_per_class)

model = SVC(kernel='linear')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:
## this is not working as of now

import numpy as np
import matplotlib.pyplot as plt
import os

def rgb2gray(image):
    return 0.2989 * image[:, :, 0] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 2]

def convolve(image, kernel):
    img_height, img_width = image.shape
    kernel_height, kernel_width = kernel.shape
    output = np.zeros_like(image)
    padded_image = np.pad(image, ((1, 1), (1, 1)), mode='constant') 
    
    for i in range(1, img_height + 1):
        for j in range(1, img_width + 1):
            region = padded_image[i-1:i+2, j-1:j+2]
            output[i-1, j-1] = np.sum(region * kernel)  
    
    return output

def sobel_edge_detection(image):
    if len(image.shape) == 3:
        image = rgb2gray(image)
    
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])

    sobel_y = np.array([[-1, -2, -1],
                        [ 0,  0,  0],
                        [ 1,  2,  1]])
    
    grad_x = convolve(image, sobel_x)
    grad_y = convolve(image, sobel_y)
    
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    return gradient_magnitude

training_folder = r'Dataset\training'
subfolders = [os.path.normpath(os.path.join(training_folder, folder)) for folder in os.listdir(training_folder) if os.path.isdir(os.path.join(training_folder, folder))]

print(subfolders)

fig, axes = plt.subplots(len(subfolders), 2, figsize=(10, 5 * len(subfolders)))
for subfolder in subfolders:
    print(f"Checking folder: {subfolder}")
    
    files = os.listdir(subfolder)
    print(f"Files in {subfolder}: {files}")
    
    image_files = [f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg', '.tif'))]
    
    if not image_files:
        print(f"No image files found in {subfolder}")
    else:
        print(f"Found image files in {subfolder}: {image_files}")
